In [ ]:
# Set random seed for reproducibility
import yfinance as yf
import numpy as np
np.random.seed(42)

# Define the list of assets and date range
assets = ['MAHABANK.NS', 'SBIN.NS', 'YESBANK.NS', 'ASHIANA.NS', 'OMAXE.NS', 'TEXINFRA.NS',
          'ADFFOODS.NS', 'BRITANNIA.NS', 'HERITGFOOD.NS', 'DONEAR.NS', 'RUPA.NS', 'KITEX.NS',
          'DALMIASUG.NS', 'UGARSUGAR.NS', 'KOTARISUG.NS', 'ONMOBILE.NS', 'HFCL.NS', 'IDEA.NS',
          'SHOPERSTOP.NS', 'V2RETAIL.NS', 'TRENT.NS', 'FORTIS.NS', 'LYKALABS.NS', 'MANGALAM.NS',
          'JPPOWER.NS', 'NHPC.NS', 'SUZLON.NS', 'ASHOKLEY.NS', 'HINDMOTORS.NS', 'TATAMTRDVR.NS']
start_date = '2017-01-01'
end_date = '2022-01-01'

# Fetch historical data
data = yf.download(assets, start=start_date, end=end_date)

# Check available columns
print("Available columns:", data.columns)

# Use 'Adj Close' if available, otherwise fallback to 'Close'
if 'Adj Close' in data.columns:
    data = data['Adj Close']
elif 'Close' in data.columns:
    data = data['Close']
else:
    raise ValueError("No valid price data found. Check asset symbols or API limits.")

# Check if data is empty
if data.empty:
    raise ValueError("Data download failed. Please check asset symbols or API limits.")

# Calculate daily returns
returns = data.pct_change().dropna()

# Calculate expected annual returns and covariance matrix
expected_returns = returns.mean().to_numpy() * 252  # Convert to numpy array
cov_matrix = returns.cov().to_numpy() * 252  # Convert to numpy array

# Define parameters for IFA
K = 20
lam = 0.5
epsilon = 0.05
gamma = 0.95
num_fireflies = 50
max_generations = 100
alpha = 0.2  # Randomness coefficient
beta_min = 0.2  # Minimum attractiveness
gamma_coef = 1.0  # Absorption coefficient

def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights

def portfolio_return(weights, returns):
    return np.sum(weights * returns)

def objective_function(weights, returns, cov_matrix, lam):
    var = portfolio_variance(weights, cov_matrix)
    ret = portfolio_return(weights, returns)
    return lam * var - (1 - lam) * ret

def move_firefly(firefly_i, firefly_j, beta, alpha):
    return firefly_i + beta * (firefly_j - firefly_i) + alpha * (np.random.rand(len(firefly_i)) - 0.5)

def improved_firefly_algorithm(num_fireflies, max_generations, K, cov_matrix, returns, lam, epsilon, gamma, alpha, beta_min, gamma_coef):
    num_assets = len(returns)

    # Initialize fireflies randomly
    fireflies = np.random.rand(num_fireflies, num_assets)
    for i in range(num_fireflies):
        idx = np.argsort(fireflies[i])[:K]
        fireflies[i, idx] = np.random.uniform(epsilon, gamma, K)
        fireflies[i, np.setdiff1d(np.arange(num_assets), idx)] = 0
        fireflies[i] /= fireflies[i].sum()

    best_solution = None
    best_value = float('inf')

    for generation in range(max_generations):
        for i in range(num_fireflies):
            for j in range(num_fireflies):
                if objective_function(fireflies[i], returns, cov_matrix, lam) > objective_function(fireflies[j], returns, cov_matrix, lam):
                    r = np.linalg.norm(fireflies[i] - fireflies[j])
                    beta = beta_min * np.exp(-gamma_coef * r ** 2)
                    fireflies[i] = move_firefly(fireflies[i], fireflies[j], beta, alpha)

                    # Ensure the cardinality constraint
                    idx = np.argsort(fireflies[i])[:K]
                    fireflies[i, idx] = np.random.uniform(epsilon, gamma, K)
                    fireflies[i, np.setdiff1d(np.arange(num_assets), idx)] = 0
                    fireflies[i] /= fireflies[i].sum()

        # Update the best solution found
        for i in range(num_fireflies):
            value = objective_function(fireflies[i], returns, cov_matrix, lam)
            if value < best_value:
                best_value = value
                best_solution = fireflies[i]

    return best_solution

# Run the Improved Firefly Algorithm
optimal_weights = improved_firefly_algorithm(num_fireflies, max_generations, K, cov_matrix, expected_returns, lam, epsilon, gamma, alpha, beta_min, gamma_coef)

# Calculate the optimal portfolio's annual return and risk
optimal_annual_return = portfolio_return(optimal_weights, expected_returns)
optimal_annual_risk = portfolio_variance(optimal_weights, cov_matrix)

print("Optimal Weights:", optimal_weights)
print("Optimal Annual Return:", optimal_annual_return)
print("Optimal Annual Risk:", optimal_annual_risk)


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  30 of 30 completed


Available columns: MultiIndex([( 'Close',   'ADFFOODS.NS'),
            ( 'Close',    'ASHIANA.NS'),
            ( 'Close',   'ASHOKLEY.NS'),
            ( 'Close',  'BRITANNIA.NS'),
            ( 'Close',  'DALMIASUG.NS'),
            ( 'Close',     'DONEAR.NS'),
            ( 'Close',     'FORTIS.NS'),
            ( 'Close', 'HERITGFOOD.NS'),
            ( 'Close',       'HFCL.NS'),
            ( 'Close', 'HINDMOTORS.NS'),
            ...
            ('Volume',       'RUPA.NS'),
            ('Volume',       'SBIN.NS'),
            ('Volume', 'SHOPERSTOP.NS'),
            ('Volume',     'SUZLON.NS'),
            ('Volume', 'TATAMTRDVR.NS'),
            ('Volume',   'TEXINFRA.NS'),
            ('Volume',      'TRENT.NS'),
            ('Volume',  'UGARSUGAR.NS'),
            ('Volume',   'V2RETAIL.NS'),
            ('Volume',    'YESBANK.NS')],
           names=['Price', 'Ticker'], length=150)
Optimal Weights: [0.09468783 0.01270607 0.06649325 0.08218215 0.07119172 0.02009441
 0.       